In [3]:
%%time
%load_ext autoreload
%autoreload 2

Wall time: 221 ms


In [94]:
import sys
from io import StringIO
import numpy as np
import pandas as pd
import requests
import datetime

### DATA SOURCES

In [95]:
# Get current date- will work at market hours
current_date = datetime.date.today().isoformat()
date = int(current_date.replace('-', ''))
date

20200812

In [96]:
date = 20200811 # default test timestamp

In [97]:
# Central Bank of Brazil: https://www.bcb.gov.br/

currencies_prices_url = f"https://www4.bcb.gov.br/Download/fechamento/{date}.csv"
currencies_info_url = f"https://www4.bcb.gov.br/Download/fechamento/M{date}.csv"

### PARSING ALL  CURRENCIES VALUES FILE

In [98]:
# downloads the csv data and parse into a pandas dataframe
data_from_url = StringIO(requests.get(currencies_prices_url).text)
cols = ['Timestamp', 'Cod Moeda', 'Tipo', 'Moeda', 'Taxa Compra', 'Taxa Venda', 'Paridade Compra', 'Paridade Venda'] 
df = pd.read_csv(data_from_url, sep=";", names=cols) 
df.head()


,Timestamp,Cod Moeda,Tipo,Moeda,Taxa Compra,Taxa Venda,Paridade Compra,Paridade Venda
0,11/08/2020,5,A,AFN,"0,07031000","0,07078000","76,70000000","77,20000000"
1,11/08/2020,9,A,ETB,"0,15020000","0,15290000","35,49440000","36,13150000"
2,11/08/2020,15,A,THB,"0,17450000","0,17450000","31,10000000","31,11000000"
3,11/08/2020,20,A,PAB,"5,42790000","5,42850000","1,00000000","1,00000000"
4,11/08/2020,27,A,VES,"0,00001910","0,00001910","283889,76530000","284601,26850000"


### PREPROCESSING CURRENCIES VALUES  DATA

In [99]:
# fixing the currenct type A and B formula by setting all rows referencing dollar

In [100]:
aux = df.set_index('Moeda')
usd = aux.loc['USD']
usd

Timestamp          11/08/2020
Cod Moeda                 220
Tipo                        A
Taxa Compra        5,42790000
Taxa Venda         5,42850000
Paridade Compra    1,00000000
Paridade Venda     1,00000000
Name: USD, dtype: object

In [101]:
usd_value = float(usd['Taxa Compra'].replace(',','.'))
usd_value

5.4279

In [102]:
# Casting strings to float
df["Taxa Compra"] = df["Taxa Compra"].str.replace(',','.')
df["Paridade Compra"] = df["Paridade Compra"].str.replace(',','.')
df.head(2)

,Timestamp,Cod Moeda,Tipo,Moeda,Taxa Compra,Taxa Venda,Paridade Compra,Paridade Venda
0,11/08/2020,5,A,AFN,0.07031000,"0,07078000",76.70000000,"77,20000000"
1,11/08/2020,9,A,ETB,0.15020000,"0,15290000",35.49440000,"36,13150000"


In [103]:
# new column with USD to currency formula
new_df = df.copy(deep=True)
new_df['USD to Currency'] = usd_value / (new_df['Taxa Compra'].astype(float))

# validating
currencies = ["USD", "EUR", "CAD"]

In [104]:
# OLD/ORIGINAL DF
df.loc[new_df['Moeda'].isin(currencies)]

,Timestamp,Cod Moeda,Tipo,Moeda,Taxa Compra,Taxa Venda,Paridade Compra,Paridade Venda
34,11/08/2020,165,A,CAD,4.08170000,"4,08250000",1.32970000,"1,32980000"
47,11/08/2020,220,A,USD,5.42790000,"5,42850000",1.00000000,"1,00000000"
154,11/08/2020,978,B,EUR,6.38320000,"6,38610000",1.17600000,"1,17640000"


In [105]:
# NEW DF
new_df.loc[new_df['Moeda'].isin(currencies)]

,Timestamp,Cod Moeda,Tipo,Moeda,Taxa Compra,Taxa Venda,Paridade Compra,Paridade Venda,USD to Currency
34,11/08/2020,165,A,CAD,4.08170000,"4,08250000",1.32970000,"1,32980000",1.329814
47,11/08/2020,220,A,USD,5.42790000,"5,42850000",1.00000000,"1,00000000",1.000000
154,11/08/2020,978,B,EUR,6.38320000,"6,38610000",1.17600000,"1,17640000",0.850342


In [108]:
# SORTING BY CURRENCY
sorted_df = new_df.sort_values(by='USD to Currency', ascending=False)
sorted_df.head(5)

,Timestamp,Cod Moeda,Tipo,Moeda,Taxa Compra,Taxa Venda,Paridade Compra,Paridade Venda,USD to Currency
4,11/08/2020,27,A,VES,0.00001910,"0,00001910",283889.76530000,"284601,26850000",284183.246073
125,11/08/2020,815,A,IRR,0.00012920,"0,00012930",42000.00000000,"42000,00000000",42011.609907
53,11/08/2020,260,A,VND,0.00023420,"0,00023420",23174.00000000,"23176,00000000",23176.345004
30,11/08/2020,148,A,STD,0.00025760,"0,00025850",21000.00000000,"21070,00000000",21071.040373
137,11/08/2020,865,A,IDR,0.00036820,"0,00037130",14620.00000000,"14740,00000000",14741.716458


### PARSING CURRENCY INFO FILE

In [109]:
cols = ['Codigo', 'Nome', 'Simbolo', 'Cod. Pais', 'Pais', 'Tipo', 'Data Exlusao Ptax'] 
df_info = pd.read_csv('data/M20200810.csv', sep=";", names=cols, encoding = "latin") 

#removes first row
df_info = df_info.iloc[1:]
df_info.head()

,Codigo,Nome,Simbolo,Cod. Pais,Pais,Tipo,Data Exlusao Ptax
1,005,AFEGANE AFEGANIST,AFN,132,AFEGANISTAO ...,A,NaN
2,785,RANDE/AFRICA SUL,ZAR,7560,AFRICA DO SUL ...,A,NaN
3,490,LEK ALBANIA REP,ALL,175,"ALBANIA, REPUBLICA DA ...",A,NaN
4,610,MARCO ALEMAO,DEM,230,ALEMANHA ...,A,02/01/2002
5,978,EURO,EUR,230,ALEMANHA ...,B,NaN


### PREPROCESSING INFO FILE

In [20]:
# FIXING THE EMPTY SPACES FROM 'Simbolo' column, as it will be our index matching criteria

#a = df_info.loc[df_info['Simbolo'] == 'EUR']; a
df_info['Simbolo'] = df_info['Simbolo'].astype(str)
df_info.iloc[1]['Simbolo']

'ZAR    '

In [21]:
df_info["Simbolo"] = df_info["Simbolo"].str.replace(' ','')
df_info.iloc[1]['Simbolo']

'ZAR'

In [22]:
currencies = ["USD", "EUR", "CAD"]
df_info.loc[df_info['Simbolo'].isin(currencies)][:10]

,Codigo,Nome,Simbolo,Cod. Pais,Pais,Tipo,Data Exlusao Ptax
5,978,EURO,EUR,230,ALEMANHA ...,B,NaN
19,978,EURO,EUR,728,AUSTRIA ...,B,NaN
28,978,EURO,EUR,876,BELGICA ...,B,NaN
50,165,DOLAR CANADENSE,CAD,1490,CANADA ...,A,NaN
61,978,EURO,EUR,1635,CHIPRE ...,B,NaN
89,978,EURO,EUR,2470,"ESLOVACA, REPUBLICA ...",B,NaN
91,978,EURO,EUR,2461,"ESLOVENIA, REPUBLICA DA ...",B,NaN
92,978,EURO,EUR,2453,ESPANHA ...,B,NaN
94,220,DOLAR DOS EUA,USD,2496,ESTADOS UNIDOS ...,A,NaN
96,978,EURO,EUR,2518,"ESTONIA, REPUBLICA DA ...",B,NaN


### JOINING THE DATAFRAMES

In [23]:
#sorted_df.merge(df_info, left_on='Moeda', right_on='Simbolo')
idx_sorted = sorted_df.set_index('Moeda')
print("\nALL VALUES DF")
idx_sorted.head(3)


ALL VALUES DF


,Timestamp,Cod Moeda,Tipo,Taxa Compra,Taxa Venda,Paridade Compra,Paridade Venda,Paridade Compra USD
Moeda,,,,,,,,
VES,10/08/2020,27,A,0.00001940,"0,00001940",277343.55110000,"278038,64770000",277974.226804
IRR,10/08/2020,815,A,0.00012840,"0,00012840",42000.00000000,"42000,00000000",41999.221184
VND,10/08/2020,260,A,0.00023260,"0,00023280",23170.00000000,"23185,00000000",23184.436801


In [24]:
idx_info = df_info.set_index('Simbolo')
print("\nINFO DF ")
idx_info.head(3)


INFO DF 


,Codigo,Nome,Cod. Pais,Pais,Tipo,Data Exlusao Ptax
Simbolo,,,,,,
AFN,005,AFEGANE AFEGANIST,132,AFEGANISTAO ...,A,NaN
ZAR,785,RANDE/AFRICA SUL,7560,AFRICA DO SUL ...,A,NaN
ALL,490,LEK ALBANIA REP,175,"ALBANIA, REPUBLICA DA ...",A,NaN


In [25]:
merge_df = idx_sorted.join(idx_info, lsuffix='_left', on='Moeda')
merge_df

,Timestamp,Cod Moeda,Tipo_left,Taxa Compra,Taxa Venda,Paridade Compra,Paridade Venda,Paridade Compra USD,Codigo,Nome,Cod. Pais,Pais,Tipo,Data Exlusao Ptax
Moeda,,,,,,,,,,,,,,
VES,10/08/2020,27,A,0.00001940,"0,00001940",277343.55110000,"278038,64770000",277974.226804,027,BOLIVAR SOBERANO VENEZUELANO,8508,VENEZUELA ...,A,NaN
IRR,10/08/2020,815,A,0.00012840,"0,00012840",42000.00000000,"42000,00000000",41999.221184,815,"RIAL/IRAN, REP",3727,"IRA, REPUBLICA ISLAMICA DO ...",A,NaN
VND,10/08/2020,260,A,0.00023260,"0,00023280",23170.00000000,"23185,00000000",23184.436801,260,DONGUE/VIETNAN,8583,VIETNA ...,A,NaN
STD,10/08/2020,148,A,0.00025590,"0,00025680",21000.00000000,"21070,00000000",21073.466198,148,DOBRA S TOME PRIN,7200,"SAO TOME E PRINCIPE, ILHAS ...",A,NaN
IDR,10/08/2020,865,A,0.00036710,"0,00036970",14590.00000000,"14690,00000000",14690.002724,865,RUPIA/INDONESIA,3654,INDONESIA ...,A,NaN
UZS,10/08/2020,893,A,0.00052550,"0,00052860",10202.60000000,"10262,60000000",10262.036156,893,SOM UZBEQUISTAO,8478,"UZBEQUISTAO, REPUBLICA DO ...",A,NaN
SLL,10/08/2020,500,A,0.00055030,"0,00055540",9710.59000000,"9800,00000000",9799.563874,500,LEONE/SERRA LEOA,7358,SERRA LEOA ...,A,NaN
GNF,10/08/2020,398,A,0.00055590,"0,00056270",9585.00000000,"9700,00000000",9700.845476,398,FRANCO/GUINE,3298,GUINE ...,A,NaN
LAK,10/08/2020,780,A,0.00059190,"0,00059770",9022.92000000,"9110,46000000",9110.829532,780,"QUIPE/LAOS, REP",4200,"LAOS, REP.POP.DEMOCR.DO ...",A,NaN


### PREPROCESSING FINAL MERGED DATAFRAME

In [26]:
currencies = ["USD", "EUR", "CAD"]
a = merge_df.loc[currencies]
a[:10]

,Timestamp,Cod Moeda,Tipo_left,Taxa Compra,Taxa Venda,Paridade Compra,Paridade Venda,Paridade Compra USD,Codigo,Nome,Cod. Pais,Pais,Tipo,Data Exlusao Ptax
Moeda,,,,,,,,,,,,,,
USD,10/08/2020,220,A,5.39270000,"5,39330000",1.00000000,"1,00000000",1.000000,220,DOLAR DOS EUA,2496,ESTADOS UNIDOS ...,A,NaN
USD,10/08/2020,220,A,5.39270000,"5,39330000",1.00000000,"1,00000000",1.000000,220,DOLAR DOS EUA,4766,"MARSHALL,ILHAS ...",A,NaN
USD,10/08/2020,220,A,5.39270000,"5,39330000",1.00000000,"1,00000000",1.000000,220,DOLAR DOS EUA,6114,PORTO RICO ...,A,NaN
USD,10/08/2020,220,A,5.39270000,"5,39330000",1.00000000,"1,00000000",1.000000,220,DOLAR DOS EUA,8630,"VIRGENS,ILHAS (BRITANICAS) ...",A,NaN
USD,10/08/2020,220,A,5.39270000,"5,39330000",1.00000000,"1,00000000",1.000000,220,DOLAR DOS EUA,8664,"VIRGENS,ILHAS (E.U.A.) ...",A,NaN
EUR,10/08/2020,978,B,6.33860000,"6,34140000",1.17540000,"1,17580000",0.850771,978,EURO,230,ALEMANHA ...,B,NaN
EUR,10/08/2020,978,B,6.33860000,"6,34140000",1.17540000,"1,17580000",0.850771,978,EURO,728,AUSTRIA ...,B,NaN
EUR,10/08/2020,978,B,6.33860000,"6,34140000",1.17540000,"1,17580000",0.850771,978,EURO,876,BELGICA ...,B,NaN
EUR,10/08/2020,978,B,6.33860000,"6,34140000",1.17540000,"1,17580000",0.850771,978,EURO,1635,CHIPRE ...,B,NaN


In [27]:
# Removing non-countries tickets, unnecessary collumns and duplicates currencies

In [28]:
final_df = merge_df.copy(deep=True)
final_df.shape

(219, 14)

In [29]:
final_cols = ['Nome','Cod Moeda', 'Pais', 'Taxa Compra', 'Paridade Compra USD']
final_df_aux = final_df[final_cols]
final_df_aux[:10]

,Nome,Cod Moeda,Pais,Taxa Compra,Paridade Compra USD
Moeda,,,,,
VES,BOLIVAR SOBERANO VENEZUELANO,27,VENEZUELA ...,0.00001940,277974.226804
IRR,"RIAL/IRAN, REP",815,"IRA, REPUBLICA ISLAMICA DO ...",0.00012840,41999.221184
VND,DONGUE/VIETNAN,260,VIETNA ...,0.00023260,23184.436801
STD,DOBRA S TOME PRIN,148,"SAO TOME E PRINCIPE, ILHAS ...",0.00025590,21073.466198
IDR,RUPIA/INDONESIA,865,INDONESIA ...,0.00036710,14690.002724
UZS,SOM UZBEQUISTAO,893,"UZBEQUISTAO, REPUBLICA DO ...",0.00052550,10262.036156
SLL,LEONE/SERRA LEOA,500,SERRA LEOA ...,0.00055030,9799.563874
GNF,FRANCO/GUINE,398,GUINE ...,0.00055590,9700.845476
LAK,"QUIPE/LAOS, REP",780,"LAOS, REP.POP.DEMOCR.DO ...",0.00059190,9110.829532


In [30]:
final_df_aux.isnull().sum()

Nome                   0
Cod Moeda              0
Pais                   7
Taxa Compra            0
Paridade Compra USD    0
dtype: int64

In [31]:
filtered_df = final_df_aux[final_df_aux['Pais'].notnull()]
filtered_df.isnull().sum()

Nome                   0
Cod Moeda              0
Pais                   0
Taxa Compra            0
Paridade Compra USD    0
dtype: int64

In [32]:
# ONLY COUNTRIES CURRENCIES ( NOT COMMODITIES, ETC ..)
filtered_df.shape

(212, 5)

In [33]:
# ONLY UNIQUE CURRENCIES ( NOT REDUNDAT COUNTRIES WITH SHARED CURRENCY, eg. EURO, ETC)
unique_filtered_df = filtered_df.drop_duplicates(subset=['Cod Moeda'])
unique_filtered_df.shape

(154, 5)

### FINAL DF

In [34]:
unique_filtered_df

,Nome,Cod Moeda,Pais,Taxa Compra,Paridade Compra USD
Moeda,,,,,
VES,BOLIVAR SOBERANO VENEZUELANO,27,VENEZUELA ...,0.00001940,277974.226804
IRR,"RIAL/IRAN, REP",815,"IRA, REPUBLICA ISLAMICA DO ...",0.00012840,41999.221184
VND,DONGUE/VIETNAN,260,VIETNA ...,0.00023260,23184.436801
STD,DOBRA S TOME PRIN,148,"SAO TOME E PRINCIPE, ILHAS ...",0.00025590,21073.466198
IDR,RUPIA/INDONESIA,865,INDONESIA ...,0.00036710,14690.002724
UZS,SOM UZBEQUISTAO,893,"UZBEQUISTAO, REPUBLICA DO ...",0.00052550,10262.036156
SLL,LEONE/SERRA LEOA,500,SERRA LEOA ...,0.00055030,9799.563874
GNF,FRANCO/GUINE,398,GUINE ...,0.00055590,9700.845476
LAK,"QUIPE/LAOS, REP",780,"LAOS, REP.POP.DEMOCR.DO ...",0.00059190,9110.829532


## CURRENCY RANK

### TOP 10 - MOST VALUABLE - EXPENSIVE -  ( 1 USD TO CURRENCY)

In [35]:
# BY COUNTRIES
aux = filtered_df[-10:]
aux.iloc[::-1]

,Nome,Cod Moeda,Pais,Taxa Compra,Paridade Compra USD
Moeda,,,,,
CLF,UNID FOMENTO CHIL,197,CHILE ...,193.66260000,0.027846
CLF,UNIDADE DE FOMENTO DO CHILE,197,CHILE ...,193.66260000,0.027846
KWD,DINAR/KWAIT,100,COVEITE ...,17.60020000,0.306400
BHD,DINAR/BAHREIN,105,"BAHREIN, ILHAS ...",14.30420000,0.377001
OMR,RIAL/OMA,805,OMA ...,14.00340000,0.385099
JOD,DINAR/JORDANIA,125,JORDANIA ...,7.59540000,0.709996
GBP,LIBRA ESTERLINA,540,REINO UNIDO ...,7.05420000,0.764467
GBP,LIBRA ESTERLINA,540,"MAN, ILHA DE ...",7.05420000,0.764467
GBP,LIBRA ESTERLINA,540,"JERSEY, ILHA DO CANAL ...",7.05420000,0.764467


In [36]:
# BY CURRENCY
aux2 = unique_filtered_df[-10:]
aux2.iloc[::-1]

,Nome,Cod Moeda,Pais,Taxa Compra,Paridade Compra USD
Moeda,,,,,
CLF,UNIDADE DE FOMENTO DO CHILE,197,CHILE ...,193.66260000,0.027846
KWD,DINAR/KWAIT,100,COVEITE ...,17.60020000,0.306400
BHD,DINAR/BAHREIN,105,"BAHREIN, ILHAS ...",14.30420000,0.377001
OMR,RIAL/OMA,805,OMA ...,14.00340000,0.385099
JOD,DINAR/JORDANIA,125,JORDANIA ...,7.59540000,0.709996
GBP,LIBRA ESTERLINA,540,"GUERNSEY, ILHA DO CANAL (INCLUI ALDERNEY E SAR...",7.05420000,0.764467
SHP,LIBRA/STA HELENA,570,SANTA HELENA ...,7.04610000,0.765345
FKP,LIBRA/FALKLAND,545,FALKLAND (ILHAS MALVINAS) ...,7.04290000,0.765693
GIP,LIBRA/GIBRALTAR,530,GIBRALTAR ...,7.03750000,0.766281


### TOP 10 - LEAST VALUABLE - CHEAP - ( 1 USD TO CURRENCY)

In [37]:
# BY COUNTRIES
filtered_df[:10]

,Nome,Cod Moeda,Pais,Taxa Compra,Paridade Compra USD
Moeda,,,,,
VES,BOLIVAR SOBERANO VENEZUELANO,27,VENEZUELA ...,0.00001940,277974.226804
IRR,"RIAL/IRAN, REP",815,"IRA, REPUBLICA ISLAMICA DO ...",0.00012840,41999.221184
VND,DONGUE/VIETNAN,260,VIETNA ...,0.00023260,23184.436801
STD,DOBRA S TOME PRIN,148,"SAO TOME E PRINCIPE, ILHAS ...",0.00025590,21073.466198
IDR,RUPIA/INDONESIA,865,INDONESIA ...,0.00036710,14690.002724
UZS,SOM UZBEQUISTAO,893,"UZBEQUISTAO, REPUBLICA DO ...",0.00052550,10262.036156
SLL,LEONE/SERRA LEOA,500,SERRA LEOA ...,0.00055030,9799.563874
GNF,FRANCO/GUINE,398,GUINE ...,0.00055590,9700.845476
LAK,"QUIPE/LAOS, REP",780,"LAOS, REP.POP.DEMOCR.DO ...",0.00059190,9110.829532


In [38]:
# BY CURRENCY
unique_filtered_df[:10]

,Nome,Cod Moeda,Pais,Taxa Compra,Paridade Compra USD
Moeda,,,,,
VES,BOLIVAR SOBERANO VENEZUELANO,27,VENEZUELA ...,0.00001940,277974.226804
IRR,"RIAL/IRAN, REP",815,"IRA, REPUBLICA ISLAMICA DO ...",0.00012840,41999.221184
VND,DONGUE/VIETNAN,260,VIETNA ...,0.00023260,23184.436801
STD,DOBRA S TOME PRIN,148,"SAO TOME E PRINCIPE, ILHAS ...",0.00025590,21073.466198
IDR,RUPIA/INDONESIA,865,INDONESIA ...,0.00036710,14690.002724
UZS,SOM UZBEQUISTAO,893,"UZBEQUISTAO, REPUBLICA DO ...",0.00052550,10262.036156
SLL,LEONE/SERRA LEOA,500,SERRA LEOA ...,0.00055030,9799.563874
GNF,FRANCO/GUINE,398,GUINE ...,0.00055590,9700.845476
LAK,"QUIPE/LAOS, REP",780,"LAOS, REP.POP.DEMOCR.DO ...",0.00059190,9110.829532
